In [1]:
from project_lib import *

In [2]:
df = pd.read_csv('dataset_tmc.csv', parse_dates = ['Dates'], date_format = '%d/%m/%y')
df.columns = [i.lower() for i in df.columns]
df.columns = [i.replace(' ','_') for i in df.columns]
df = df.set_index('dates')

# Check NaNs
if df.isna().sum().sum() != 0:
    print('check NaNs in data')

# Drop Sat&Sun if any
df = df[df.index.dayofweek<5]

# as convention 1 week is 5 trading days and 1 month is 21 trading days so that each year has 252 trading days


In [12]:
# STRATEGY PARAMETERS
TRADED_SECURITIES = [
            'us_staples',
            # 'eu_healthcare', 'japan', 'brazil', 'nasdaq', 'us_2y',
            # 'us_15y', 'germany_10y', 'uk_15y', 'us_hy', 'oil', 'ind_metals', 'agri',
            # 'gold', 'silver', 'eurusd', 'usdjpy', 'chfjpy', 'eurbrl', 'gbpmxn'
    ]
LONG_PARAMS = {10:0.05, 21:0.15, 63:0.25} # dict with lenght of window as key and percentile for longs as values
SHORT_PARAMS = {10:0.95, 21:0.85, 63:0.75} # dict with lenght of window as key and percentile for shorts as values
MA_FAST_WDW = 20
MA_SLOW_WDW = 60
SUPPORTIVE_PCTL_MOVE = 0.3
COUNTER_PCTL_MOVE = 0.2
MAX_DOLLAR_LOSS = 100


In [39]:
for security_id in TRADED_SECURITIES:
    # generate signal based on standard strategy
    signal_all = meanrev_signal(df[security_id],
                                long_params = LONG_PARAMS,
                                short_params = SHORT_PARAMS,
                                ma_fast_wdw = MA_FAST_WDW,
                                ma_slow_wdw = MA_SLOW_WDW
                )

    # "start your backtest at t-10". hard coding initial date
    signal_all = signal_all.loc['2014-02-12':]

    # extract from all signal only the actual open buy/sell triggers
    signal_do = signal_all[signal_all!= 0]

    # list to store all trades
    trades_list = []

    # iterates over all buy/sell signal and execute orders
    for dt_open, direction in zip(signal_do.index, signal_do):
        
        # create a unique id for each trade
        trade_id = security_id+'#'+str(dt_open)[:10]

        # price at which the trade is open
        price_open = df.loc[dt_open, security_id]

        # compute TP/SL returns and prices
        tp_return, sl_return = tp_sl_rule(df[security_id],
                                            dt_open,
                                            direction,
                                            supportive_pctl_move = SUPPORTIVE_PCTL_MOVE,
                                            counter_pctl_move = COUNTER_PCTL_MOVE
                                          )
        if direction == 1:
            price_tp = price_open * (1+tp_return)
            price_sl = price_open * (1+sl_return)
        elif direction == -1:
            price_tp = price_open * (1 - tp_return)
            price_sl = price_open * (1 - sl_return)
        # compute the optimal sizing such that all trades loses the same amount of $ if SL is hitted
        quantity = MAX_DOLLAR_LOSS/((price_open - price_sl)*direction)

        trade = pd.DataFrame([security_id, dt_open, price_open, direction, quantity, price_tp, price_sl],
                     columns=[trade_id],
                     index=['security_id', 'dt_open', 'price_open', 'direction', 'quantity', 'price_tp', 'price_sl']).T
        
        trades_list.append(trade)

        

        




In [42]:
pd.concat(trades_list)

,security_id,dt_open,price_open,direction,quantity,price_tp,price_sl
us_staples#2014-03-04,us_staples,2014-03-04 00:00:00,42.69,-1,202.680699,42.213262,43.183387
us_staples#2014-03-10,us_staples,2014-03-10 00:00:00,42.7,-1,195.314722,42.22315,43.211994
us_staples#2014-03-12,us_staples,2014-03-12 00:00:00,42.77,-1,194.662956,42.292368,43.283708
us_staples#2014-03-17,us_staples,2014-03-17 00:00:00,42.83,-1,193.5426,42.351698,43.346682
us_staples#2014-03-18,us_staples,2014-03-18 00:00:00,42.92,-1,192.623436,42.440693,43.439148
...,...,...,...,...,...,...,...
us_staples#2023-12-29,us_staples,2023-12-29 00:00:00,72.03,-1,140.193833,71.252166,72.743298
us_staples#2024-01-02,us_staples,2024-01-02 00:00:00,72.84,-1,138.44434,72.053419,73.562312
us_staples#2024-01-29,us_staples,2024-01-29 00:00:00,73.05,-1,138.141259,72.261151,73.773897
us_staples#2024-01-30,us_staples,2024-01-30 00:00:00,73.46,-1,137.275874,72.666724,74.18846


In [43]:
signal_do

dates
2014-03-04   -1
2014-03-10   -1
2014-03-12   -1
2014-03-17   -1
2014-03-18   -1
             ..
2023-12-29   -1
2024-01-02   -1
2024-01-29   -1
2024-01-30   -1
2024-02-01   -1
Length: 549, dtype: int64